In [ ]:
import sys
!{sys.executable} -m pip install torchviz

In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
import ppscore as pps

import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import warnings
import itertools
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.stattools import adfuller

from torchviz import make_dot # cool library to visualize neural network chart


from pandas_profiling import ProfileReport

C:\Users\vince\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
path_data ="C:/PROJET/CRYPTO/data/"

In [4]:



# Ignore the warning, can comment this 
warnings.filterwarnings("ignore")

## loading data
path_data = "C:/PROJET/CRYPTO/data/"


with open("C:/PROJET/CRYPTO/data/yahoo_finance/10_best_crypto_pickle", "rb") as f:
    df_multidx = pickle.load(f)

with open("C:/PROJET/CRYPTO/data/yahoo_finance/pickle_dict_crypto_stocks", "rb") as f:
    df_dict = pickle.load(f)

In [5]:
# if we want to create a new simple (aka non multiindex) dataframe based on "open" value

# choose the value we want to select from, here 'Open' columns for each crypto
df_open = df_multidx.iloc[:,df_multidx.columns.get_level_values(0) ==  "Open"]

# then drop the level we want to suppres ("0"th level which correspond to "Open")
df_open.columns= df_open.columns.droplevel(0)

# we now have a simple dataframe that contains all the "Open" values
df_open

,BCH-USD,BNB-USD,BSV-USD,BTC-USD,EOS-USD,ETH-USD,LTC-USD,USDT-USD,XRP-USD,XTZ-USD
Date,,,,,,,,,,
2014-09-17,NaN,NaN,NaN,465.864014,NaN,NaN,5.085890,NaN,0.005123,NaN
2014-09-18,NaN,NaN,NaN,456.859985,NaN,NaN,5.065430,NaN,0.005398,NaN
2014-09-19,NaN,NaN,NaN,424.102997,NaN,NaN,4.687290,NaN,0.005134,NaN
2014-09-20,NaN,NaN,NaN,394.673004,NaN,NaN,4.329200,NaN,0.004942,NaN
2014-09-21,NaN,NaN,NaN,408.084991,NaN,NaN,4.263070,NaN,0.004768,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-11-03,257.199005,27.883926,158.671066,13550.451172,2.430642,383.156036,53.817410,1.000523,0.235420,1.896543
2020-11-04,244.146500,26.857388,153.782562,13950.488281,2.361036,387.603210,53.819469,1.000552,0.239510,1.868321
2020-11-05,241.142044,26.931576,152.169510,14133.733398,2.349351,402.142944,54.501873,1.001079,0.237799,1.876342


In [6]:
df_btc = df_dict["BTC-USD"]

In [7]:
# if we want to create a new simple (aka non multiindex) dataframe based on "open" value

# choose the value we want to select from, here 'Open' columns for each crypto
df_vol = df_multidx.iloc[:,df_multidx.columns.get_level_values(0) ==  "Volume"]

# then drop the level we want to suppres ("0"th level which correspond to "Open")
df_vol.columns= df_vol.columns.droplevel(0)

# we now have a simple dataframe that contains all the "Open" values
df_vol

,BCH-USD,BNB-USD,BSV-USD,BTC-USD,EOS-USD,ETH-USD,LTC-USD,USDT-USD,XRP-USD,XTZ-USD
Date,,,,,,,,,,
2014-09-17,NaN,NaN,NaN,21056800,NaN,NaN,3071840,NaN,1281960,NaN
2014-09-18,NaN,NaN,NaN,34483200,NaN,NaN,4569260,NaN,491147,NaN
2014-09-19,NaN,NaN,NaN,37919700,NaN,NaN,3917450,NaN,407109,NaN
2014-09-20,NaN,NaN,NaN,36863600,NaN,NaN,5490660,NaN,322621,NaN
2014-09-21,NaN,NaN,NaN,26580100,NaN,NaN,2931220,NaN,309571,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-11-03,2.902353e+09,457901612.0,826433367.0,29869951617,2.498280e+09,1.258849e+10,2580300878,4.501917e+10,3052524907,138243205.0
2020-11-04,2.602973e+09,383972479.0,965812020.0,35116364961,2.195202e+09,1.512608e+10,3050534139,5.135149e+10,3335786073,161150029.0
2020-11-05,2.296535e+09,299236755.0,741458841.0,40856321438,2.271829e+09,1.544071e+10,3250514598,5.036133e+10,2949508994,206184470.0


In [8]:
profile_open = df_open.profile_report(title =  "10 best crytpo opening prices")
profile_open.to_widgets()

In [9]:
profile_open.to_file( path_data +"10 best crytpo opening prices.html")

In [ ]:
profile_volume= df_open.profile_report(title =  "10 best crytpo volume")
profile_volume.to_widgets()

In [ ]:
profile_volume.to_file( path_data +"10 best crytpo volume.html")

In [ ]:
df_open.columns


In [10]:
df_open  = df_open[['BCH-USD',
 'BNB-USD',
 'BSV-USD',
 'EOS-USD',
 'ETH-USD',
 'LTC-USD',
 'USDT-USD',
 'XRP-USD',
 'XTZ-USD', 'BTC-USD']]
# df_vol = df_vol[['BCH-USD',
#  'BNB-USD',
#  'BSV-USD',
#  'EOS-USD',
#  'ETH-USD',
#  'LTC-USD',
#  'USDT-USD',
#  'XRP-USD',
#  'XTZ-USD', 'BTC-USD']]

In [11]:
df_open.dtypes

BCH-USD     float64
BNB-USD     float64
BSV-USD     float64
EOS-USD     float64
ETH-USD     float64
LTC-USD     float64
USDT-USD    float64
XRP-USD     float64
XTZ-USD     float64
BTC-USD     float64
dtype: object

In [12]:
y_col = df_open.columns[-1]
x_col = df_open.columns[:-1].to_list()
x_col

['BCH-USD',
 'BNB-USD',
 'BSV-USD',
 'EOS-USD',
 'ETH-USD',
 'LTC-USD',
 'USDT-USD',
 'XRP-USD',
 'XTZ-USD']

In [ ]:
df_open.to_csv(path_data + "df_open.csv", index= 0)
df_vol.to_csv(path_data + "df_vol.csv", index= 0)

In [ ]:
df =pd.read_csv(path_data + "df_open.csv")
df

In [ ]:
pps.score(df_open, x = x_col, y= "BTC-USD")

In [ ]:
import seaborn as sns
import ppscore as pps

#predictors_df = pps.predictors(df, y="y")
fig, ax = plt.subplots(figsize=(15,10))

predictors_df = pps.predictors(df_open, y= "BTC-USD")
sns.barplot(data=predictors_df, x=x_col, y="ppscore", ax =ax)

print(predictors_df)


In [ ]:
matrix = pps.matrix(df_open)[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))


sns.heatmap(matrix, vmin=0, vmax=1, 
            cmap="PuBu", 
            linewidths=0.5, 
            annot=True,
            ax=ax)

In [ ]:
matrix1 = pps.matrix(df_vol)[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

sns.heatmap(matrix1, vmin=0, vmax=1, 
            cmap="Blues", 
            linewidths=0.5, 
            annot=True,
            ax=ax)

## loading hourly data 

In [13]:
import yfinance as yf # https://github.com/ranaroussi/yfinance


In [14]:
#load hourly data
def scrap_yahoo_fin(stock_list, start, end, period =None, interval = 1, prepost = False,auto_adjust = True, actions =True ):
    """
        stock_list = either a str typed ticker (ex: "dxy") or a list (ex: ['DXY', "INX"])
        period: data period to download (Either Use period parameter or use start and end) Valid periods are: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
        interval: data interval (intraday data cannot extend last 60 days) Valid intervals are: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
        start: If not using period - Download start date string (YYYY-MM-DD) or datetime.
        end: If not using period - Download end date string (YYYY-MM-DD) or datetime.
        prepost: Include Pre and Post market data in results? (Default is False)
        auto_adjust: Adjust all OHLC automatically? (Default is True)
        actions: Download stock dividends and stock splits events? (Default is True)
    """
    data = yf.download(stocks, start=start, end=end)
    return data


In [15]:
#stocks = ["BTC-USD","ETH-USD", "XRP-USD", "USDT-USD", "BCH-USD", "BSV-USD", "LTC-USD", "EOS-USD", "BNB-USD", "XTZ-USD"]
stocks = ["BTC-USD",
          #"LTC-USD", 
          #"XRP-USD"
         ]
# start and en perid comes form date index of min and max  historical data of bitcoin EUR
start ="2015-01-01" 
end ="2020-11-20"

df =yf.download(stocks, start=start, end=end, interval = "1d")

[*********************100%***********************]  1 of 1 completed


get data prepared for pytorch (#2)

source: Deep Learning for Time Series Forecasting https://github.com/0215Arthur/deep-learning-time-series-forecasting

https://github.com/Azure/DeepLearningForTimeSeriesForecasting

https://github.com/zhangxu0307/time_series_forecasting_pytorch (!!!!!)

to check https://github.com/Haoran-Zhao/Deep-Learning-for-Time-Series-Forecasting (???)
deep learnign ressource and code regarding time series(!!!!!) https://github.com/Alro10/deep-learning-time-series

code examples:
#1 Encoder-Decoder Model for Multistep Time Series Forecasting Using PyTorch
https://github.com/gautham20/pytorch-ts
https://towardsdatascience.com/encoder-decoder-model-for-multistep-time-series-forecasting-using-pytorch-5d54c6af6e60


#https://github.com/zhangxu0307/time_series_forecasting_pytorch/blob/master/src/model.py


In [16]:
def dtype_convertor(df, dtype):
    temp =df.columns
    for element in temp:
        df[element] = df[element].astype(dtype, errors = "ignore")
#     if df[element] !+
#         pass
#         return df

In [17]:
dtype_convertor(df_btc, "float32")
#dtype_convertor(df_vol, "float32")


In [18]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)


In [19]:
get_device()

device(type='cuda', index=0)

## CONV RNN model

source
https://github.com/KurochkinAlexey/ConvRNN/blob/master/ConvRNN_SML2010.ipynb

In [20]:
["BTC-USD","LTC-USD", "XRP-USD"]

['BTC-USD', 'LTC-USD', 'XRP-USD']

In [21]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-12-31,310.914001,320.192993,310.210999,320.192993,320.192993,13942900
2015-01-01,320.434998,320.434998,314.002991,314.248993,314.248993,8036550
2015-01-02,314.079010,315.838989,313.565002,315.032013,315.032013,7860650
2015-01-03,314.846008,315.149994,281.082001,281.082001,281.082001,33054400
2015-01-04,281.145996,287.230011,257.612000,264.195007,264.195007,55629100
...,...,...,...,...,...,...
2020-11-15,16068.139648,16123.110352,15793.534180,15955.587891,15955.587891,23653867582
2020-11-16,15955.577148,16816.181641,15880.706055,16716.111328,16716.111328,31526766675
2020-11-17,16685.691406,17782.919922,16564.544922,17645.406250,17645.406250,39006849169


In [22]:
 # choosing dat only without nan values
df = df_open.loc[:,["BTC-USD"]]

In [23]:
df

,BTC-USD
Date,
2014-09-17,465.864014
2014-09-18,456.859985
2014-09-19,424.102997
2014-09-20,394.673004
2014-09-21,408.084991
...,...
2020-11-03,13550.451172
2020-11-04,13950.488281
2020-11-05,14133.733398


In [26]:
df = df.reset_index(inplace =False)


In [27]:
df

,index,Date,BTC-USD
0,0,2014-09-17,465.864014
1,1,2014-09-18,456.859985
2,2,2014-09-19,424.102997
3,3,2014-09-20,394.673004
4,4,2014-09-21,408.084991
...,...,...,...
2239,2239,2020-11-03,13550.451172
2240,2240,2020-11-04,13950.488281
2241,2241,2020-11-05,14133.733398
2242,2242,2020-11-06,15579.729492


In [28]:
#df_new1, df_new2 = df[:, :], df[10:, :] 
data1, data2 = df.iloc[:1122,:],df.iloc[1122:,:]

In [29]:
df.columns

Index(['index', 'Date', 'BTC-USD'], dtype='object')

In [ ]:
target = "Close"
cols= ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

# target = "BTC-USD"
# cols= ["LTC-USD", "XRP-USD"]

#cols = ['BCH-USD', 'BNB-USD', 'BSV-USD', 'EOS-USD', 'ETH-USD', 'LTC-USD','USDT-USD', 'XRP-USD', 'XTZ-USD', 'BTC-USD']

In [ ]:
# splitting dataset in 2 to fit the code

data1 =df.iloc[:1122,:]
data2 = df.iloc[1122:,:]
#data2 = df_open.tail(a)

In [ ]:
train_size = 2000
val_size = 244
depth = 90
batch_size = 128
prediction_horizon = 1

In [ ]:
X_train1 = np.zeros((len(data1), depth, len(cols)))
y_train1 = np.zeros((len(data1), 1))


In [ ]:
for i, name in enumerate(cols):
    for j in range(depth):
        X_train1[:, j, i] = data1[name].shift(depth - j - 1).fillna(method="bfill")
y_train1 = data1[target].shift(-prediction_horizon).fillna(method='ffill')

In [ ]:
X_train1 = X_train1[depth:-prediction_horizon]
y_train1 = y_train1[depth:-prediction_horizon]

In [ ]:
X2 = np.zeros((len(data2), depth, len(cols)))
y2 = np.zeros((len(data2), 1))

In [ ]:
for i, name in enumerate(cols):
    for j in range(depth):
        X2[:, j, i] = data2[name].shift(depth - j - 1).fillna(method="bfill")
y2 = data2[target].shift(-prediction_horizon).fillna(method='ffill')

In [ ]:
X_train2 = X2[:train_size - len(data1)]
y_train2 = y2[:train_size - len(data1)]

X_val = X2[train_size - len(data1):train_size - len(data1) + val_size]
y_val = y2[train_size - len(data1):train_size - len(data1) + val_size]

X_test = X2[train_size - len(data1) + val_size:]
y_test = y2[train_size - len(data1) + val_size:]

In [ ]:
X_train2 = X_train2[depth:]
y_train2 = y_train2[depth:]

In [ ]:
X_train = np.concatenate([X_train1, X_train2], axis=0)
y_train = np.concatenate([y_train1, y_train2], axis=0)

In [ ]:
X_train.shape, y_train.shape


In [ ]:
X_train

In [ ]:

X_train_min, y_train_min = X_train.min(axis=0), y_train.min(axis=0)
X_train_max, y_train_max = X_train.max(axis=0), y_train.max(axis=0)

In [ ]:

X_train = (X_train - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_val = (X_val - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_test = (X_test - X_train_min)/(X_train_max - X_train_min + 1e-9)

y_train = (y_train - y_train_min)/(y_train_max - y_train_min + 1e-9)
y_val = (y_val - y_train_min)/(y_train_max - y_train_min + 1e-9)
y_test = (y_test - y_train_min)/(y_train_max - y_train_min + 1e-9)

In [ ]:
X_train_t = torch.Tensor(X_train)
X_val_t = torch.Tensor(X_val)
X_test_t = torch.Tensor(X_test)
y_train_t = torch.Tensor(y_train)
y_val_t = torch.Tensor(y_val.values)
y_test_t = torch.Tensor(y_test.values)

In [ ]:
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), shuffle=True, batch_size=batch_size)
val_loader = DataLoader(TensorDataset(X_val_t, y_val_t), shuffle=False, batch_size=batch_size)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), shuffle=False, batch_size=batch_size)

In [ ]:
class ConvRNN(nn.Module):
    def __init__(self, input_dim, timesteps, output_dim, kernel_size1=7, kernel_size2=5, kernel_size3=3, 
                 n_channels1=32, n_channels2=32, n_channels3=32, n_units1=32, n_units2=32, n_units3=32):
        super().__init__()
        self.avg_pool1 = nn.AvgPool1d(2, 2)
        self.avg_pool2 = nn.AvgPool1d(4, 4)
        self.conv11 = nn.Conv1d(input_dim, n_channels1, kernel_size=kernel_size1)
        self.conv12 = nn.Conv1d(n_channels1, n_channels1, kernel_size=kernel_size1)
        self.conv21 = nn.Conv1d(input_dim, n_channels2, kernel_size=kernel_size2)
        self.conv22 = nn.Conv1d(n_channels2, n_channels2, kernel_size=kernel_size2)
        self.conv31 = nn.Conv1d(input_dim, n_channels3, kernel_size=kernel_size3)
        self.conv32 = nn.Conv1d(n_channels3, n_channels3, kernel_size=kernel_size3)
        self.gru1 = nn.GRU(n_channels1, n_units1, batch_first=True)
        self.gru2 = nn.GRU(n_channels2, n_units2, batch_first=True)
        self.gru3 = nn.GRU(n_channels3, n_units3, batch_first=True)
        self.linear1 = nn.Linear(n_units1+n_units2+n_units3, output_dim)
        self.linear2 = nn.Linear(input_dim*timesteps, output_dim)
        self.zp11 = nn.ConstantPad1d(((kernel_size1-1), 0), 0)
        self.zp12 = nn.ConstantPad1d(((kernel_size1-1), 0), 0)
        self.zp21 = nn.ConstantPad1d(((kernel_size2-1), 0), 0)
        self.zp22 = nn.ConstantPad1d(((kernel_size2-1), 0), 0)
        self.zp31 = nn.ConstantPad1d(((kernel_size3-1), 0), 0)
        self.zp32 = nn.ConstantPad1d(((kernel_size3-1), 0), 0)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        # line1
        y1 = self.zp11(x)
        y1 = torch.relu(self.conv11(y1))
        y1 = self.zp12(y1)
        y1 = torch.relu(self.conv12(y1))
        y1 = y1.permute(0, 2, 1)
        out, h1 = self.gru1(y1)
        # line2
        y2 = self.avg_pool1(x)
        y2 = self.zp21(y2)
        y2 = torch.relu(self.conv21(y2))
        y2 = self.zp22(y2)
        y2 = torch.relu(self.conv22(y2))
        y2 = y2.permute(0, 2, 1)
        out, h2 = self.gru2(y2)
        # line3 
        y3 = self.avg_pool2(x)
        y3 = self.zp31(y3)
        y3 = torch.relu(self.conv31(y3))
        y3 = self.zp32(y3)
        y3 = torch.relu(self.conv32(y3))
        y3 = y3.permute(0, 2, 1)
        out, h3 = self.gru3(y3)
        h = torch.cat([h1[-1], h2[-1], h3[-1]], dim=1)
        out1 = self.linear1(h)
        out2 = self.linear2(x.contiguous().view(x.shape[0], -1))
        out = out1 + out2
        return out

In [ ]:
model = ConvRNN(X_train.shape[2], X_train.shape[1], 1, n_channels1=128, n_channels2=128, n_channels3=128,
                n_units1=128, n_units2=128, n_units3=128).cuda()
opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epoch_scheduler = torch.optim.lr_scheduler.StepLR(opt, 20, gamma=0.9)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [ ]:
import time


In [ ]:
epochs = 500
loss = nn.MSELoss()
patience = 75
min_val_loss = 9999
counter = 0
for i in range(epochs):
    mse_train = 0
    iteration_start = time.monotonic()
    for batch_x, batch_y in train_loader :
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        opt.zero_grad()
        y_pred = model(batch_x)
        y_pred = y_pred.squeeze(1)
        l = loss(y_pred, batch_y)
        l.backward()
        mse_train += l.item()*batch_x.shape[0]
        opt.step()
    epoch_scheduler.step()
    with torch.no_grad():
        mse_val = 0
        preds = []
        true = []
        for batch_x, batch_y in val_loader:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()
            output = model(batch_x)
            output = output.squeeze(1)
            preds.append(output.detach().cpu().numpy())
            true.append(batch_y.detach().cpu().numpy())
            mse_val += loss(output, batch_y).item()*batch_x.shape[0]
    preds = np.concatenate(preds)
    true = np.concatenate(true)
    
    if min_val_loss > mse_val**0.5:
        min_val_loss = mse_val**0.5
        print("Saving...")
        torch.save(model.state_dict(), "convrnn_sml2010.pt")
        counter = 0
    else: 
        counter += 1
    
    if counter == patience:
        break
    print("Iter: ", i, "train: ", (mse_train/len(X_train_t))**0.5, "val: ", (mse_val/len(X_val_t))**0.5)
    iteration_end = time.monotonic()
    print("Iter time: ", iteration_end - iteration_start)
    if(i % 10 == 0):
        preds = preds*(y_train_max - y_train_min) + y_train_min
        true = true*(y_train_max - y_train_min) + y_train_min
        mse = mean_squared_error(true, preds)
        mae = mean_absolute_error(true, preds)
        print("mse: ", mse, "mae: ", mae)
        plt.figure(figsize=(20, 10))
        plt.plot(preds)
        plt.plot(true)
        plt.show()

In [ ]:
model.load_state_dict(torch.load("convrnn_sml2010.pt"))


In [ ]:
preds

In [ ]:
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item()*batch_x.shape[0]
# preds = np.concatenate(preds)
# true = np.concatenate(true)

In [ ]:
print(listest_loader)

In [ ]:
preds = preds*(y_train_max - y_train_min) + y_train_min
true = true*(y_train_max - y_train_min) + y_train_min

In [ ]:

mse = mean_squared_error(true, preds)
mae = mean_absolute_error(true, preds)

In [ ]:
mse, mae

In [ ]:
mse**0.5


In [ ]:

plt.figure(figsize=(20, 10))
plt.plot(preds)
plt.plot(true)
plt.show()

## Stock Price Prediction with PyTorch
https://medium.com/swlh/stock-price-prediction-with-pytorch-37f52ae84632

In [ ]:
df_btc

In [ ]:
data = df_btc.reset_index(inplace =False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

price = data[['Close']]
scaler = MinMaxScaler(feature_range=(-1, 1))

price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))

In [ ]:
def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]
lookback = 60 # choose sequence length
x_train, y_train, x_test, y_test = split_data(price, lookback)

In [ ]:
import torch
import torch.nn as nn
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [ ]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 300

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
from torchviz import make_dot
a = model(x_train)
make_dot(a)

In [ ]:
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

In [ ]:
predict = pd.DataFrame(scaler.inverse_transform(y_train_pred.detach().numpy()))
original = pd.DataFrame(scaler.inverse_transform(y_train_lstm.detach().numpy()))

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")    

fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot(1, 2, 1)
ax = sns.lineplot(x = original.index, y = original[0], label="Data", color='royalblue')
ax = sns.lineplot(x = predict.index, y = predict[0], label="Training Prediction (LSTM)", color='tomato')
ax.set_title('Stock price', size = 14, fontweight='bold')
ax.set_xlabel("Days", size = 14)
ax.set_ylabel("Cost (USD)", size = 14)
ax.set_xticklabels('', size=10)


plt.subplot(1, 2, 2)
ax = sns.lineplot(data=hist, color='royalblue')
ax.set_xlabel("Epoch", size = 14)
ax.set_ylabel("Loss", size = 14)
ax.set_title("Training Loss", size = 14, fontweight='bold')
fig.set_figheight(6)
fig.set_figwidth(16)

In [ ]:
type(y_test_lstm)

In [ ]:
scaler.inverse_transform(y_train_pred)

In [ ]:
x=T.ones(10, requires_grad=True)

y=x**2
z=x**3
r=(y+z).sum()
make_dot(r).render("attached", format="png")

In [ ]:
# make predictions
import math, time
y_test_pred = model(x_test)

from torch.autograd import Variable

# invert predictions
    # y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
    # AttributeError: 'numpy.ndarray' object has no attribute 'detach'
y_train_pred = scaler.inverse_transform(y_train_pred)
y_train = scaler.inverse_transform(y_train_lstm.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test_lstm.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
lstm.append(trainScore)
lstm.append(testScore)
lstm.append(training_time)

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(price)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[lookback:len(y_train_pred)+lookback, :] = y_train_pred

# shift test predictions for plotting
testPredictPlot = np.empty_like(price)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(y_train_pred)+lookback-1:len(price)-1, :] = y_test_pred

original = scaler.inverse_transform(price['Close'].values.reshape(-1,1))

predictions = np.append(trainPredictPlot, testPredictPlot, axis=1)
predictions = np.append(predictions, original, axis=1)
result = pd.DataFrame(predictions)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
                    mode='lines',
                    name='Train prediction')))
fig.add_trace(go.Scatter(x=result.index, y=result[1],
                    mode='lines',
                    name='Test prediction'))
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[2],
                    mode='lines',
                    name='Actual Value')))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=False,
        linecolor='white',
        linewidth=2
    ),
    yaxis=dict(
        title_text='Close (USD)',
        titlefont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='white',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
    ),
    showlegend=True,
    template = 'plotly_dark'

)



annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Results (LSTM)',
                              font=dict(family='Rockwell',
                                        size=26,
                                        color='white'),
                              showarrow=False))
fig.update_layout(annotations=annotations)

fig.show()

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

predict = pd.DataFrame(scaler.inverse_transform(y_train_pred.detach().numpy()))
original = pd.DataFrame(scaler.inverse_transform(y_train_lstm.detach().numpy()))

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")    

fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot(1, 2, 1)
ax = sns.lineplot(x = original.index, y = original[0], label="Data", color='royalblue')
ax = sns.lineplot(x = predict.index, y = predict[0], label="Training Prediction (LSTM)", color='tomato')
ax.set_title('Stock price', size = 14, fontweight='bold')
ax.set_xlabel("Days", size = 14)
ax.set_ylabel("Cost (USD)", size = 14)
ax.set_xticklabels('', size=10)


plt.subplot(1, 2, 2)
ax = sns.lineplot(data=hist, color='royalblue')
ax.set_xlabel("Epoch", size = 14)
ax.set_ylabel("Loss", size = 14)
ax.set_title("Training Loss", size = 14, fontweight='bold')
fig.set_figheight(6)
fig.set_figwidth(16)

In [ ]:
# make predictions
import math, time
y_test_pred = model(x_test)

from torch.autograd import Variable

# invert predictions
    # y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
    # AttributeError: 'numpy.ndarray' object has no attribute 'detach'
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train_lstm.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test_lstm.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
lstm.append(trainScore)
lstm.append(testScore)
lstm.append(training_time)



In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(price)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[lookback:len(y_train_pred)+lookback, :] = y_train_pred

# shift test predictions for plotting
testPredictPlot = np.empty_like(price)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(y_train_pred)+lookback-1:len(price)-1, :] = y_test_pred

original = scaler.inverse_transform(price['Close'].values.reshape(-1,1))

predictions = np.append(trainPredictPlot, testPredictPlot, axis=1)
predictions = np.append(predictions, original, axis=1)
result = pd.DataFrame(predictions)


In [ ]:
result

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
                    mode='lines',
                    name='Train prediction')))

fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[2],
                    mode='lines',
                    name='Actual Value')))
fig.add_trace(go.Scatter(x=result.index, y=result[1],
                    mode='lines',
                    line_color = "blue",
                    name='Test prediction'))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=False,
        linecolor='white',
        linewidth=2
    ),
    yaxis=dict(
        title_text='Close (USD)',
        titlefont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='white',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
    ),
    showlegend=True,
    template = 'plotly_dark'

)



annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Results (LSTM)',
                              font=dict(family='Rockwell',
                                        size=26,
                                        color='white'),
                              showarrow=False))
fig.update_layout(annotations=annotations)

fig.show()